In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
import re
import json
import random
import time

In [2]:
# get trail html
def get_soup(url):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}
    page = get(url,headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

In [3]:
missed_trails = pd.read_csv('/Users/briangraham/insight/trailrec/data/missed_trail_urls')

In [4]:
randomized_trail_list_full = pd.read_csv('/Users/briangraham/insight/trailrec/data/random_scrap_list')

In [6]:
missed_trails.head()

,Unnamed: 0,missed_trail_url
0,0,https://www.trailforks.com/trails/capilano-pac...
1,1,https://www.trailforks.com/trails/hiking-loop-...
2,2,https://www.trailforks.com/trails/burrill-aven...
3,3,https://www.trailforks.com/trails/orchard-uppe...
4,4,https://www.trailforks.com/trails/boiler-maker...


In [7]:
randomized_trail_list_full.head()

,Unnamed: 0,Unnamed: 0.1,trail_id,trail_url,trail_name,region_url,region_name
0,784,784,brat-22073,https://www.trailforks.com/trails/brat-22073/,BRAT,https://www.trailforks.com/region/sechelt/,Sechelt
1,3817,3817,meadow-of-the-grizzly-spelhx-en-tl-a-stl-lhale...,https://www.trailforks.com/trails/meadow-of-th...,Meadow of the Grizzly (Spélhx̱en tl'a Stl'lhalem),https://www.trailforks.com/region/diamond-head/,Diamond Head
2,6262,6262,the-dungeon,https://www.trailforks.com/trails/the-dungeon/,The Dungeon,https://www.trailforks.com/region/iron-mountain/,Iron Mountain
3,6538,6538,treacherous-cretins,https://www.trailforks.com/trails/treacherous-...,Treacherous Cretins,https://www.trailforks.com/region/lost-lake/,Lost Lake
4,979,979,capilano-pacific-trail-exit,https://www.trailforks.com/trails/capilano-pac...,Capilano Pacific Trail Exit,https://www.trailforks.com/region/west-vancouver/,West Vancouver


In [8]:
missed_trails = pd.read_csv('/Users/briangraham/insight/trailrec/data/missed_trail_urls')
randomized_trail_list_full = pd.read_csv('/Users/briangraham/insight/trailrec/data/random_scrap_list')
merged = pd.merge(missed_trails,randomized_trail_list_full, 
         left_on='missed_trail_url', right_on='trail_url', how='outer', indicator=True)
len(merged[merged['_merge'] == 'right_only'])
merged_clipped = merged[merged['_merge'] == 'right_only']

In [9]:
merged_clipped[merged_clipped['trail_id'] == 'bear-claw'].head()

,Unnamed: 0_x,missed_trail_url,Unnamed: 0_y,Unnamed: 0.1,trail_id,trail_url,trail_name,region_url,region_name,_merge
6969,NaN,NaN,410,410,bear-claw,https://www.trailforks.com/trails/bear-claw/,Bear Claw,https://www.trailforks.com/region/buntzen-lake...,Buntzen Lake Recreation Area,right_only


In [10]:
merged = merged_clipped.loc[6969:]

In [18]:
test_url = merged['trail_url'][0] + 'ridelogs/'
soup  = get_soup('https://www.trailforks.com/trails/verboten/ridelogs/')

KeyError: 0

In [15]:
ride_date = []
rider_url = []
rider_name = []
rider_state = []
rider_city = []
ride_difficulty_string = []

table = soup.find('section').find('table',id='').find('tbody').find_all('tr')
for row in table:
    row_struct = row.find_all('td')
    ride_date.append(row_struct[1].find('span').text)
    rider_url.append(row_struct[2].find('a').attrs['href'])
    rider_name.append(row_struct[2].find('a').text)
    rider_state.append(row_struct[4].text)
    rider_city.append(row_struct[5].text)
    ride_difficulty_string.append(row_struct[8].span.attrs['title'])

NameError: name 'soup' is not defined

In [11]:
def get_page_url_list(urlbase):
    trail_list_soup = get_soup(urlbase) 
    check_log = trail_list_soup.find('ul', class_='paging-middle centertext')
    url_list = []
    if check_log:
        pagedata = check_log.find_all('li')
        lastpage = int(pagedata[-1].text)
        
        for i in np.arange(lastpage):
            n = i+1
            if n == 1:
                url = urlbase
            else:
                url = urlbase + '?page=' + str(n)
            url_list.append(url)
        return url_list
    else:
        return url_list
    

In [12]:
errorlog3 = []
for t_id,url in zip(merged['trail_id'],merged['trail_url']):
    ridelog_url = url + 'ridelogs/'
    page_list = get_page_url_list(ridelog_url)
    ride_date = []
    rider_url = []
    rider_name = []
    rider_state = []
    rider_city = []
    ride_difficulty_string = []
    try:
        for page in page_list:
            soup  = get_soup(page)
            table = soup.find('section').find('table',id='').find('tbody').find_all('tr')
            for row in table:
                row_struct = row.find_all('td')
                ride_date.append(row_struct[1].find('span').text)
                rider_url.append(row_struct[2].find('a').attrs['href'])
                rider_name.append(row_struct[2].find('a').text)
                rider_state.append(row_struct[4].text)
                rider_city.append(row_struct[5].text)
                ride_difficulty_string.append(row_struct[8].span.attrs['title'])
        ridelog_data_save_loc = '/Users/briangraham/insight/trailrec/data/ridelogs/'
        df_ridelog_data = pd.DataFrame({'ride_date':ride_date,'rider_url':rider_url,'rider_name':rider_name,'rider_state':rider_state,'rider_city':rider_city,'ride_difficulty_string':ride_difficulty_string})
        df_ridelog_data.to_pickle(ridelog_data_save_loc + t_id + '.pickle')
    except:
        errorlog3.append(url)

In [45]:
errorlog

[]